In [ ]:
import pandas as pd
import respiration.utils as utils

analysis_dir = utils.dir_path('outputs', 'analysis')

metrics_file = utils.join_paths(analysis_dir, 'metrics.csv')
metrics = pd.read_csv(metrics_file)

metrics_avg_file = utils.join_paths(analysis_dir, 'metrics_average.csv')
metrics_average = pd.read_csv(metrics_avg_file)

frequencies_file = utils.join_paths(analysis_dir, 'frequencies.csv')
frequencies = pd.read_csv(frequencies_file)

In [ ]:
figure_dir = utils.dir_path('outputs', 'figures', mkdir=True)

In [ ]:
groups = [
    {
        'label': 'Optical Flow',
        'models': [
            'raft_small',
            'FlowNet2CS',
            'lucas_kanade',
            'pixel_intensity_rgb',
        ],
    },
    {
        'label': 'rPPG',
        'models': [
            'MMPD_intra_RhythmFormer',
            'UBFC-rPPG_TSCAN',
            'UBFC-rPPG_EfficientPhys',
            'BP4D_PseudoLabel_DeepPhys',
        ],
    },
    {
        'label': 'R-RhythmFormer',
        'models': [
            'RF_20240902_210159',
        ],
    },
    {
        'label': 'SimpleViT',
        'models': [
            'SimpleViT_20240821_115511',
        ],
    },
    {
        'label': 'Pretrained',
        'models': [
            'mtts_can',
            'big_small',
        ],
    },
    {
        'label': 'Random',
        'models': [
            'random',
        ],
    },
]

In [ ]:
# Plot the MAE and PCC for the psd method
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 6))

points = []

for group in groups:
    group_points = metrics_average[metrics_average['model'].isin(group['models'])]

    for _, row in group_points.iterrows():
        points.append({
            'model': row['model'],
            'mae': row['mae'],
            'pcc': row['pcc'],
            'group': group['label'],
        })

points = pd.DataFrame(points)

# Scatter the MAE and PCC for the different models
sns.scatterplot(
    data=points,
    x='mae',
    y='pcc',
    s=250,
    style='model',
    hue='group',
)

plt.xlabel('MAE (BPM)')
plt.ylabel('Correlation')
plt.title('MAE and Pearson Correlation for the different models')
plt.tight_layout()

# Set the dimensions of the plot
plt.xlim(0, 8)
plt.ylim(0, 1)

# Place the legend outside the plot
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
import numpy as np
from scipy import stats

group_records = []

for group1 in groups:
    for group2 in groups:
        # if group1 == group2:
        #     continue

        group1_points = frequencies[frequencies['model'].isin(group1['models'])]
        group2_points = frequencies[frequencies['model'].isin(group2['models'])]

        error_1 = np.abs(group1_points['prediction'] - group1_points['ground_truth'])
        error_2 = np.abs(group2_points['prediction'] - group2_points['ground_truth'])

        mae_1 = error_1.mean()
        mae_2 = error_2.mean()

        t_stat, p = stats.ttest_ind(error_1, error_2)

        group_records.append({
            'group1': group1['label'],
            'group2': group2['label'],
            'mae_1': round(mae_1 * 60, 1),
            'mae_2': round(mae_2 * 60, 1),
            't_stat': round(t_stat, 3),
            'p_value': round(p, 3),
        })

group_records = pd.DataFrame(group_records)
group_records

In [ ]:
# Visualize the t_stat in a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))

# Create a pivot table
heatmap = group_records.pivot_table(
    index='group1',
    columns='group2',
    values='t_stat',
)

# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(heatmap)

sns.heatmap(
    data=heatmap,
    annot=True,
    fmt='.0f',
    # cmap='viridis',
    mask=matrix,
)

# Turn the label on the y-axis
plt.yticks(rotation=0)

plt.title('Group T-Statistics')
plt.tight_layout()
plt.show()